In [1]:
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
import deap
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
x = pd.DataFrame(iris.data, columns=[iris.feature_names])
y = pd.Series(iris.target)

In [2]:
x.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
y

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Length: 150, dtype: int32

In [4]:
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.30)

In [5]:
x_treino.shape

(105, 4)

In [6]:
# Criando o modelo da rede neural (sem informar compilador ou algoritmo otimizador, apenas o esqueleto da rede):
# Essa rede terá 4 neurônios de entrada (estado atual), 10 neurônios na 1ª camada oculta, 5 neurônios na segunda e 3 neurônios
# na saída: 
modelo = Sequential()
modelo.add(Dense(10, input_dim=4, activation='relu'))
modelo.add(Dense(5, activation='relu'))
modelo.add(Dense(3, activation='softmax')) 

In [7]:
# Função que recebe 2 dados: uma lista de pesos da rede neural e uma lista com as dimensões da rede neural (no seguinte formato):
# [n_input, n_neuronios_1a_camada_oculta, n_neuronios_2a_camada_oculta, ..., n_output] e retorna uma lista com os pesos da rede 
# no formato que o Keras espera receber. 

# Observe que, se a rede tem dimensões 10 x 30 x 30 x 4, significa que possui 10*30 pesos + 30 bias referentes à primeira camada, 
# 30*30 + 30 na segunda camada, etc.
def organiza_pesos(pesos, dimensoes_rede):

    pesos_para_keras=[]
    desloca=0
    
    for camada in range(len(dimensoes_rede)-1): # para cada dimensão da rede, exceto a saída
        weights=[]
        for neuronio in range(dimensoes_rede[camada]): # irá rodar x vezes para cada camada da rede, sendo x o nº de neurônios de cada camada
            weights.append(pesos[desloca:desloca+dimensoes_rede[camada+1]]) # coleta várias listas do tamanho da próxima camada (como resultado teremos, por exemplo: 10 x 30)
            desloca=desloca+dimensoes_rede[camada+1]
            
        pesos_para_keras.append(np.array(weights)) # transformando a lista de listas acima em um array numppy
        pesos_para_keras.append(np.array(pesos[desloca:desloca+dimensoes_rede[camada+1]])) # adicionando os bias como array numpy
        desloca=desloca+dimensoes_rede[camada+1]

    return pesos_para_keras

# A variável pesos_para_keras será uma lista de arrays, onde cada array consiste em um conjunto de pesos ou bias referentes a
# cada camada. 
# No nosso exemplo, teríamos um array com 10x30 elementos, depois um array com 1x30 elementos (bias), depois um array com 30x30
# elementos, etc. 

In [8]:
# Se a rede fosse 10 x 30 x 30 x 4, a quantidade de pesos total seria: 10x30 + 30 + 30x30 + 30 + 30x4 + 4 = 1384
vetor_pesos = [0 for i in range(1384)]
pesos_keras = organiza_pesos(vetor_pesos, [10, 30, 30, 4])

In [9]:
pesos_keras

[array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0

In [10]:
# Fitness Function 
from sklearn.metrics import r2_score

def coeficiente_r2(individuo):
    modelo.set_weights(organiza_pesos(individuo,[4,10,5,3]))
    #previsoes = modelo.predict_classes(x_treino) substituido pelas 3 linhas abaixo
    #codigo abaixo baseado nos jupyter notebooks: 2022.09.15_ReconhecimentoDeFala e 2022.06.08_Keras_Iris#2
    prob = modelo.predict(x_treino)
    indice=np.argmax(prob,axis=1)
    previsoes=indice
    score = r2_score(y_treino, previsoes)
    return score,

In [11]:
# Criando o algoritmo genético:
creator.create(name="ClasseFitness", base=base.Fitness, weights=(1.0,))     
creator.create(name="ClasseIndividuos", base=list, fitness=creator.ClasseFitness) 

# Toolbox
modulo_toolbox = base.Toolbox()
modulo_toolbox.register("pesos_e_bias", random.uniform, -1, 1) # definindo pesos no range [-1, 1]                  
modulo_toolbox.register("individuos", tools.initRepeat, creator.ClasseIndividuos, modulo_toolbox.pesos_e_bias, 123)
# Aqui temos 123 variáveis por indivíduo, pois 4x10 + 10 + 10x5 + 5 + 5x3 + 3 = 123 (qtde de pesos e bias totais da rede)
modulo_toolbox.register("populacao", tools.initRepeat, list, modulo_toolbox.individuos, 100)  # criando uma população de 100 indivíduos
modulo_toolbox.register("evaluate", coeficiente_r2) # informando a Fitness Function
modulo_toolbox.register("mate", tools.cxBlend, alpha=0.5) # informando o crossover
# A função cxBlend mistura cada um dos elementos entre dois indivíduos, fazendo uma espécie de média ponderada randômica, onde 
# às vezes  o valor de um indivíduo recebe mais peso do que outro, e esses pesos podem ser positivos ou negativos. 
# Na prática, é feito o cálculo gamma*x1 + (1-gamma)*x2, onde gamma varia dentro dos limites de [-1,2] (se alpha=1) ou
# nos limites de [0,1] se alpha=0, ou [-0.5, 1.5] se alpha=0.5...
modulo_toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.2, indpb=0.05) # informando a  mutação
# A função mutGaussian adiciona um valor a cada elemento sorteado do indivíduo (tendo probabilidade 'indpb' de sortear cada elemento),
# sendo que o valor adicionado é retirado de uma curva de distribuição normal de média "mu" e desvio padrão "sigma".
modulo_toolbox.register("select", tools.selTournament, tournsize=3) # informando a seleção natural

In [12]:
# Executando o código completo em ordem:
    
populacao = modulo_toolbox.populacao() 
hof = tools.HallOfFame(1) # salvando o melhor indivíduo de todos
estatisticas = tools.Statistics(lambda ind: ind.fitness.values) # criando estatísticas de cada população

estatisticas.register("media", np.mean)
estatisticas.register("dp", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

# Rodando o algoritmo completo:
populacao, log = algorithms.eaSimple(populacao, modulo_toolbox, cxpb=0.7, mutpb=0.2, ngen=50, stats=estatisticas, halloffame=hof, verbose=True)

4/4 [==============================] - 0s 997us/step
gen	nevals	media    	dp      	min     	max     
0  	100   	-0.866729	0.817914	-3.35644	0.540695
4/4 [==============================] - 0s 2ms/step


4/4 [==============================] - 0s 1ms/step
1  	72    	-0.616337	0.747929	-1.8811 	0.526776
4/4 [==============================] - 0s 1ms/step
2  	68    	-0.507078	0.795039	-3.2451 	0.540695
4/4 [==============================] - 0s 2ms/step


4/4 [==============================] - 0s 1ms/step
3  	83    	-0.543684	0.837984	-3.46779	0.540695
4/4 [==============================] - 0s 998us/step
4  	68    	-0.544797	0.829189	-3.34252	0.526776
4/4 [==============================] - 0s 2ms/step
5  	80    	-0.616059	0.8124  	-2.07595	0.540695
4/4 [==============================] - 0s 1ms/step


4/4 [==============================] - 0s 1ms/step
6  	74    	-0.520718	0.844241	-3.43995	0.540695
4/4 [==============================] - 0s 1ms/step
7  	77    	-0.550086	0.881924	-3.46779	0.540695
4/4 [==============================] - 0s 1ms/step


4/4 [==============================] - 0s 996us/step
8  	86    	-0.446394	0.779509	-2.79971	0.540695
4/4 [==============================] - 0s 2ms/step


...

4/4 [==============================] - 0s 0s/step
40 	73    	0.741954  	0.222932	-0.405753	0.958245
4/4 [==============================] - 0s 1ms/step
41 	72    	0.691152  	0.337793	-1.3522  	0.958245
4/4 [==============================] - 0s 0s/step
42 	84    	0.680574  	0.386045	-1.57489 	0.958245
4/4 [==============================] - 0s 5ms/step


4/4 [==============================] - 0s 0s/step
43 	76    	0.74432   	0.288333	-1.3522  	0.958245
4/4 [==============================] - 0s 0s/step
44 	76    	0.787049  	0.193345	-0.266569	0.958245
4/4 [==============================] - 0s 0s/step


4/4 [==============================] - 0s 1ms/step
45 	80    	0.735551  	0.244638	-0.614528	0.958245
4/4 [==============================] - 0s 0s/step
46 	75    	0.774662  	0.237018	-0.614528	0.958245
4/4 [==============================] - 0s 5ms/step


4/4 [==============================] - 0s 0s/step
47 	79    	0.709524  	0.3544  	-1.18518 	0.958245
4/4 [==============================] - 0s 1ms/step
48 	75    	0.697137  	0.353499	-1.3522  	0.958245
4/4 [==============================] - 0s 2ms/step


4/4 [==============================] - 0s 1ms/step
49 	76    	0.677512  	0.40964 	-1.3522  	0.958245
4/4 [==============================] - 0s 1ms/step
50 	76    	0.681687  	0.437894	-1.3522  	0.958245


In [13]:
pesos_finais = hof[0]

In [14]:
pesos_finais

[-0.8316771128979529,
 -0.6549942032148324,
 -0.17718599810071958,
 0.600335363750493,
 0.568131742531397,
 0.6175259423595831,
 -0.4153854281014434,
 0.7866219104094689,
 -0.11157749795901244,
 0.08820479921454,
 -0.34111500271475703,
 -1.345440459812974,
 2.8075800990682156,
 0.1487119015445668,
 0.13544494026851225,
 0.4249826800452385,
 -0.33450352837495495,
 -1.3068740033244102,
 0.544908815988841,
 0.49312555941376596,
 0.19051862292458704,
 0.5249426749513132,
 -0.44535772935372064,
 -0.176122534237082,
 0.5047016757654186,
 -0.0983419947908423,
 0.6934343429707972,
 -1.0783932742668894,
 0.6270921050814786,
 -0.4250173711102952,
 -0.18739945714662581,
 -0.1242164964073782,
 -2.198325269450139,
 0.07210476375907757,
 0.5755349042554114,
 1.5651808945655215,
 -0.7989955293078034,
 0.5347836880450404,
 0.08121429603936586,
 -1.4064900501950208,
 0.033728507843985345,
 0.060499737157640815,
 0.5058837905057465,
 -0.5350404197857972,
 -0.6986462264162996,
 -0.5623465911943647,
 -0.2

In [15]:
from sklearn.metrics import r2_score
modelo.set_weights(organiza_pesos(pesos_finais,[4,10,5,3]))
#previsoes = modelo.predict_classes(x_teste)
prob = modelo.predict(x_teste)
prob

2/2 [==============================] - 0s 2ms/step


array([[2.74427757e-02, 3.98450524e-01, 5.74106693e-01],
       [7.79678464e-01, 2.20179588e-01, 1.41935627e-04],
       [9.12266612e-01, 8.77316967e-02, 1.72414047e-06],
       [1.69690419e-02, 2.44711071e-01, 7.38319933e-01],
       [8.50109637e-01, 1.49869412e-01, 2.09887567e-05],
       [8.68873000e-01, 1.31115988e-01, 1.10938827e-05],
       [1.81259573e-01, 6.94046497e-01, 1.24693953e-01],
       [3.75888906e-02, 3.90176684e-01, 5.72234452e-01],
       [8.47752631e-01, 1.52224749e-01, 2.26261127e-05],
       [9.13022816e-01, 8.69755074e-02, 1.65754238e-06],
       [8.67644101e-02, 5.77851534e-01, 3.35384071e-01],
       [1.00661948e-01, 6.14699900e-01, 2.84638107e-01],
       [8.74114692e-01, 1.25876054e-01, 9.15194050e-06],
       [1.50464773e-01, 6.56981528e-01, 1.92553714e-01],
       [1.92278504e-01, 6.97577357e-01, 1.10144153e-01],
       [1.05353437e-01, 6.27760887e-01, 2.66885638e-01],
       [8.01663995e-01, 1.98252872e-01, 8.31345460e-05],
       [8.24360847e-01, 1.75593

In [16]:
np.set_printoptions(formatter={'float':lambda x: "{0:0.2f}".format(x)})
prob

array([[0.03, 0.40, 0.57],
       [0.78, 0.22, 0.00],
       [0.91, 0.09, 0.00],
       [0.02, 0.24, 0.74],
       [0.85, 0.15, 0.00],
       [0.87, 0.13, 0.00],
       [0.18, 0.69, 0.12],
       [0.04, 0.39, 0.57],
       [0.85, 0.15, 0.00],
       [0.91, 0.09, 0.00],
       [0.09, 0.58, 0.34],
       [0.10, 0.61, 0.28],
       [0.87, 0.13, 0.00],
       [0.15, 0.66, 0.19],
       [0.19, 0.70, 0.11],
       [0.11, 0.63, 0.27],
       [0.80, 0.20, 0.00],
       [0.82, 0.18, 0.00],
       [0.19, 0.69, 0.12],
       [0.82, 0.18, 0.00],
       [0.18, 0.68, 0.14],
       [0.03, 0.32, 0.65],
       [0.16, 0.71, 0.13],
       [0.01, 0.20, 0.79],
       [0.01, 0.22, 0.77],
       [0.01, 0.17, 0.82],
       [0.10, 0.62, 0.28],
       [0.01, 0.16, 0.83],
       [0.01, 0.18, 0.81],
       [0.01, 0.17, 0.82],
       [0.12, 0.67, 0.21],
       [0.00, 0.13, 0.86],
       [0.00, 0.13, 0.87],
       [0.02, 0.31, 0.67],
       [0.87, 0.13, 0.00],
       [0.05, 0.48, 0.47],
       [0.00, 0.12, 0.87],
 

In [17]:
indice=np.argmax(prob,axis=1)
indice

array([2, 0, 0, 2, 0, 0, 1, 2, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 2,
       1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 0, 1, 2, 2, 1, 1, 0, 1, 1, 2,
       1], dtype=int64)

In [18]:
previsoes=indice
previsoes

array([2, 0, 0, 2, 0, 0, 1, 2, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 2,
       1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 0, 1, 2, 2, 1, 1, 0, 1, 1, 2,
       1], dtype=int64)

In [19]:
score = r2_score(y_teste, previsoes)
print("R2:", score)

R2: 0.927652733118971


In [20]:
organiza_pesos(pesos_finais,[4,10,5,3])

[array([[-0.83, -0.65, -0.18, 0.60, 0.57, 0.62, -0.42, 0.79, -0.11, 0.09],
        [-0.34, -1.35, 2.81, 0.15, 0.14, 0.42, -0.33, -1.31, 0.54, 0.49],
        [0.19, 0.52, -0.45, -0.18, 0.50, -0.10, 0.69, -1.08, 0.63, -0.43],
        [-0.19, -0.12, -2.20, 0.07, 0.58, 1.57, -0.80, 0.53, 0.08, -1.41]]),
 array([0.03, 0.06, 0.51, -0.54, -0.70, -0.56, -0.25, -0.22, 0.75, -0.58]),
 array([[0.42, -0.08, -2.16, -1.25, 0.41],
        [0.03, 0.40, 0.16, -0.23, -0.29],
        [0.47, 0.03, -0.04, -0.35, 1.05],
        [-0.86, 0.99, -0.19, -0.94, 0.97],
        [-0.18, 0.21, 0.35, 1.32, -0.63],
        [-0.70, -0.80, -0.74, -0.64, 0.42],
        [0.20, -0.28, -3.63, 1.18, -1.52],
        [0.07, 0.69, -0.04, -0.44, 0.83],
        [-0.11, -2.63, -0.11, 0.70, -0.79],
        [-0.53, 0.93, 0.77, 1.27, 1.17]]),
 array([-0.83, 0.63, 0.37, -0.11, 1.83]),
 array([[1.88, -0.27, -1.13],
        [0.41, -0.19, -1.23],
        [0.63, -1.01, 3.50],
        [0.04, 0.42, 0.78],
        [0.89, 0.66, -0.08]]),
 arra